## Aula 26 – Assembly x86 Modo Real : Modo Gráfico


# 1. Introdução

A plataforma IBM-PC prevê a memória de vídeo mapeada na memória principal. A memória de vídeo tem, em cada posição o valor da cor do pixel que deve aparecer na tela. Basicamente, a placa de vídeo é um dispositivo (ou melhor, era, pois hoje é praticamente um computador completo independente) que fica em laço infinito lendo tal trecho da memória e vai transformando cada valor lido no sinal elétrico correspondente para o monitor acender o pixel com a intensidade correta. Até hoje toda placa de vídeo para PC tem que se comportar desse jeito para ser compatível com a plataforma.

Entretanto, existem muitos modos de funcionamento do vídeo. Pode ser funcionamento exclusivo de texto, pode ser preto e branco, colorido com poucas cores, colorido com muitas cores, resoluções diferentes, etc. Assim que o computador é ligado, a placa inicia seu funcionamento em modo texto, ou seja, ela se comporta como vimos no “hello world” sem interrupção: ao invés de ler pixels na memória, ela lê texto e se responsabiliza pelo desenho dos caracteres. E se quisermos modo gráfico, ou seja, se quisermos que ela leia valores de intensidade de pixels na memória como descrito no parágrafo anterior? Temos que pedir à placa que ela mude o seu modo de operação para o modo gráfico e podemos fazer isso utilizando, novamente, a interrupção de vídeo (INT 0x10).

O programa abaixo pinta toda a tela de vermelho pixel a pixel. Faz isso colocando o valor correspondente à cor vermelha em todas as posições da memória no trecho onde está mapeado o vídeo.

     org 0x7c00
     bits 16
     
     mov ax, 0
     mov ds, ax
     cli

     mov al, 0x13
     int 0x10

     mov ax, 0xA000
     mov es, ax

     mov cx, 64000
     mov di, 0

loop:
     mov [es:di], byte 40
     inc di
     dec cx
     jnz loop

     hlt

     times 510 - ($-$$) db 0
     dw 0xaa55


Analisemos os trechos que utilizam as novidades discutidas nesta aula:

     mov al, 0x13
     int 0x10

A interrupção 0x10 é a mesma que a gente já utilizava para fazer o “print de caracteres”, pois ela é a “interrupção de vídeo”. Aqui estamos utilizando outra função dela, que é a mudança de modo de vídeo. No caso, temos AH = 0, que significa “mude o modo” e AL = “0x13” que significa “modo 0x13”. O modo 0x13, no caso, é o modo VGA de 256 cores e resolução de 320x200. Existem vários modos definidos na plataforma IBM-PC por padrão (ver leitura complementar), mas esse é o que disponibiliza mais cores e, portanto, com o qual conseguiremos os melhores resultados gráficos. Essa limitação é devido ao modo real, pois mais cores e mais resolução exigiriam mais memória.

Após a execução da interrupção, a placa de vídeo passa a ler cada byte do segmento 0xA000 como sendo a cor de um pixel em sequência na tela. Perceba que, se a resolução é de 320x200, temos 64000 pixels. Portanto, os primeiros 64000 bytes do segmento 0xA000 são as cores dos pontos na tela. Além disso, vale ressaltar também que, como cada pixel é representado por um byte, isso explica o motivo de só serem possíveis 256 cores diferentes (dá para ter mais cores utilizando alguns truques que discutiremos em aulas futuras). Logo, o restante do programa consiste em fazer um loop de 64000 repetições onde, em cada uma delas, preenchemos uma posição da memória com o valor referente à cor vermelha, partindo do índice 0 do segmento 0xA000.

	mov cx, 64000
    
     mov di, 0

loop:

     mov [es:di], byte 40
     
     inc di
     
     dec cx
     
     jnz loop

A estratégia utilizada foi utilizar o registrador CX como contador, iniciando-o com o valor 64000 e repentir o loop enquanto ele não fosse igual a zero, sendo decrementado a cada passo. Enquanto isso, utilizamos o registrador DI para marcar o índice (endereço) da memória, incrementando-o em cada passo. A cor vermelha é dada pelo byte 40, pois também é padrão do modo VGA. O modo VGA define as cores por uma “tabela de cores” (ou paleta) com tamanho 256, assim cada posição pode ser referenciada por apenas um byte. Ou seja, o byte 40 é vermelho porque na posição 40 da paleta está definida a cor vermelha. A tabela pode ser modificada, como veremos em aula futura, mas ela já inicia com seus valores padrões (vem gravado na placa de vídeo) e deixaremos para tal aula a explicação de como a cor é formada dentro da tabela. Veja a leitura complementar para conhecer as cores disponíveis por padrão e seus respectivos índices (bytes).

## 2. Leitura Complementar

https://en.wikipedia.org/wiki/Mode_13h
https://www.fountainware.com/EXPL/vga_color_palettes.htm

## 3. Exercícios

1. Escreva uma subrotina que, dados o número de uma coluna no registrador AX, de uma linha no registrador DX e de uma cor no registrador CL, seja atribuída a respectiva cor ao pixel correto. Por exemplo, se AX = 100, DX = 50 e CL = 40, o pixel (50,100) recebe a cor vermelha. Dica: como na memória os pixels estão distribuídos linearmente e que a tela tem no total 320 colunas e 200 linhas de resolução, dadas coordenadas em linha e coluna você pode definir o endereço por: end = 320*linha + coluna (assuma que linhas e colunas começam em 0).

2. Pesquise o algoritmo de “Bresenham” para desenho de linhas e, com a subrotina criada no exercício 1, implemente-o em assembly.